In [1]:
from h2o.tree import H2OTree
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2ODeepLearningEstimator
import h2o
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

In [2]:
h2o.init(min_mem_size="8G", max_mem_size="14G")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.6+9-LTS-190, mixed mode, sharing)
  Starting server from C:\Users\Eryk\PycharmProjects\webScraping\venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Eryk\AppData\Local\Temp\tmpon8ueh7d
  JVM stdout: C:\Users\Eryk\AppData\Local\Temp\tmpon8ueh7d\h2o_Eryk_started_from_python.out
  JVM stderr: C:\Users\Eryk\AppData\Local\Temp\tmpon8ueh7d\h2o_Eryk_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Warsaw
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,1 month and 19 days
H2O_cluster_name:,H2O_from_python_Eryk_9q87hb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,14 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [ ]:
df = pd.read_csv("merge_dataset.csv", sep="\t")
df.drop(columns=[
    "Miasto", 
    "URL",
    "Pietro", 
    "formatted_address",
    "result_confidence",
    "suburb",
    "building_category",
    "result_type",
    "Dataset"
], inplace=True)

In [3]:
numeric_imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
categorical_imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
binary_imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
categorical_encoder = OneHotEncoder(drop="first")

categorical_pipeline = Pipeline(
    steps=[
        ("imputer", categorical_imputer),
        ("encoder", categorical_encoder)
    ]
)
binary_pipeline = binary_imputer
numeric_pipeline = numeric_imputer

preprocessing = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, make_column_selector(dtype_include=float)),
        ("cat", categorical_pipeline, make_column_selector(dtype_include=object)),
        ("bin", binary_pipeline, make_column_selector(dtype_include=int))
    ]
)

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
preprocessor = preprocessing.fit(train.iloc[:, 1:])
num_features = list(preprocessing.named_transformers_["num"].feature_names_in_)
cat_features = list(preprocessing.named_transformers_["cat"].steps[1][1].get_feature_names_out())
bin_features = list(preprocessing.named_transformers_["bin"].feature_names_in_)
features = num_features + cat_features + bin_features

In [ ]:
train_preprocessed = pd.DataFrame(preprocessor.transform(train.iloc[:, 1:]), columns=features)
y_train = train.iloc[:, 0]
test_preprocessed = pd.DataFrame(preprocessor.transform(test.iloc[:, 1:]), columns=features)
y_test = test.iloc[:, 0]

In [ ]:
h2o_x_train = h2o.H2OFrame(train_preprocessed, column_names=features)
h2o_y_train = h2o.H2OFrame(y_train.to_numpy(), column_names=["Cena"])
h2o_train = h2o_x_train
h2o_train["Cena"] = h2o_y_train
h2o_x_test = h2o.H2OFrame(test_preprocessed, column_names=features)
h2o_y_test = h2o.H2OFrame(y_test.to_numpy(), column_names=["Cena"])
h2o_test = h2o_x_test
h2o_test["Cena"] = h2o_y_test

In [ ]:
predictors = h2o_x_train.columns
response = h2o_y_train.columns[0]

In [ ]:
response

### Baseline models

In [ ]:
model = H2ORandomForestEstimator(seed=1)
model.train(
    x = predictors,
    y = response,
    training_frame = h2o_train,
    validation_frame = h2o_test
)
print(model.model_performance())
print(model.model_performance(h2o_test))
print(model.r2())
print(model.r2(valid=True))

In [ ]:
model = H2OGradientBoostingEstimator()
model.train(
    x = predictors,
    y = response,
    training_frame = h2o_train,
    validation_frame = h2o_test
)
print(model.model_performance())
print(model.model_performance(h2o_test))
print(model.r2())
print(model.r2(valid=True))

In [ ]:
model = H2OGeneralizedLinearEstimator(solver="COORDINATE_DESCENT_NAIVE", alpha=1)
model.train(
    x = predictors,
    y = response,
    training_frame = h2o_train,
    validation_frame = h2o_test
)
print(model.model_performance())
print(model.model_performance(h2o_test))
print(model.r2())
print(model.r2(valid=True))

In [ ]:
model = H2ODeepLearningEstimator()
model.train(
    x = predictors,
    y = response,
    training_frame = h2o_x_train,
    validation_frame = h2o_x_test
)
print(model.model_performance())
print(model.model_performance(h2o_test))
print(model.r2())
print(model.r2(valid=True))

## Outlier detection and data scaling

In [4]:
df = pd.read_csv("merge_dataset.csv", sep="\t")
df.drop(columns=[
    "Miasto", 
    "URL",
    "Pietro", 
    "formatted_address",
    "result_confidence",
    "suburb",
    "building_category",
    "result_type",
    "Dataset"
], inplace=True)

In [5]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
train_outlier = train.copy()
test_outlier = test.copy()
train_outlier = train_outlier[(train_outlier["Cena"]>250000)&(train_outlier["Cena"]<4000000)]
test_outlier = test_outlier[(test_outlier["Cena"]>250000)&(test_outlier["Cena"]<4000000)]
train_outlier = train_outlier[(train_outlier["Powierzchnia"]<300)]
test_outlier = test_outlier[(test_outlier["Powierzchnia"]<300)]

In [6]:
preprocessor = preprocessing.fit(train_outlier.iloc[:, 1:])
num_features = list(preprocessing.named_transformers_["num"].feature_names_in_)
cat_features = list(preprocessing.named_transformers_["cat"].steps[1][1].get_feature_names_out())
bin_features = list(preprocessing.named_transformers_["bin"].feature_names_in_)
features = num_features + cat_features + bin_features

In [7]:
train_preprocessed = pd.DataFrame(preprocessor.transform(train_outlier.iloc[:, 1:]), columns=features)
y_train = train_outlier.iloc[:, 0]
test_preprocessed = pd.DataFrame(preprocessor.transform(test_outlier.iloc[:, 1:]), columns=features)
y_test = test_outlier.iloc[:, 0]

In [8]:
h2o_x_train = h2o.H2OFrame(train_preprocessed, column_names=features)
h2o_y_train = h2o.H2OFrame(y_train.to_numpy(), column_names=["Cena"])
h2o_train = h2o_x_train
h2o_train["Cena"] = h2o_y_train
h2o_x_test = h2o.H2OFrame(test_preprocessed, column_names=features)
h2o_y_test = h2o.H2OFrame(y_test.to_numpy(), column_names=["Cena"])
h2o_test = h2o_x_test
h2o_test["Cena"] = h2o_y_test

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
predictors = h2o_x_train.columns
response = h2o_y_train.columns[0]

### Random Forest

In [ ]:
model = H2ORandomForestEstimator(seed=1)
model.train(
    x = predictors,
    y = response,
    training_frame = h2o_train,
    validation_frame = h2o_test
)
print(model.model_performance())
print(model.model_performance(h2o_test))
print(model.r2())
print(model.r2(valid=True))

In [10]:
hyper_params = {
    "ntrees": [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
    'max_depth' : [int(x) for x in np.linspace(10, 200, num = 5)],
     "min_rows": [1, 2, 4, 8, 16],
    'sample_rate': [x/100. for x in range(20,101)],
    'nbins' : [1, 2, 6, 10, 20, 50, 100],
    "seed": [1]
}

search_criteria = { 
    "strategy":"RandomDiscrete",   # Can be also cartesian                      
    'max_runtime_secs': 14400, # 4h   14400 
    'max_models': 300,  ## build no more than 100 models
    'seed' : 1
}

rf = H2ORandomForestEstimator()
rf_grid = H2OGridSearch(
    model = rf, 
    hyper_params = hyper_params,
    search_criteria = search_criteria
)

rf_grid.train(
    x = predictors,
    y = response,
    training_frame = h2o_x_train,
    validation_frame = h2o_x_test
)

drf Grid Build progress: |██████████████Job request failed Unexpected HTTP error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), will retry after 3s.
 (failed)


OSError: Job with key $03017f00000132d4ffffffff$_ae35ba79d214de2a1eec33d0ce424d2b failed with an exception: java.lang.OutOfMemoryError: Java heap space
stacktrace: 
java.lang.OutOfMemoryError: Java heap space


In [ ]:
rf_grid.models

In [ ]:
print(rf_grid.model_performance())
print(rf_grid.model_performance(h2o_test))
print(rf_grid.r2())
print(rf_grid.r2(valid=True))

## Gradient Boosting Regressor

In [ ]:
model = H2OGradientBoostingEstimator()
model.train(
    x = predictors,
    y = response,
    training_frame = h2o_x_train,
    validation_frame = h2o_x_test
)
print(model.model_performance())
print(model.model_performance(h2o_test))
print(model.r2())
print(model.r2(valid=True))

In [ ]:
hyper_params = {
    'max_depth' : [int(x) for x in np.linspace(start = 1, stop = 200, num = 5)],
    "ntrees": [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)],
    "min_rows": [int(x) for x in np.linspace(start = 2, stop = 30, num = 1)],
    'learn_rate': [0.00001, 0.0001, 0.001, 0.01, 0.1, 0.25, 0.5, 1],
    "min_split_improvement": [0.000000001, 0.00000001, 0.0000001, 0.000001, 0.0001, 0.001, 0.01, 0.1, 0.3],
    'sample_rate': [x/100. for x in range(20,101)],
    'col_sample_rate' : [x/100. for x in range(20,101)],
    "seed": [1]
}
grid_id = 'max_depth_grid'
search_criteria = { 
    "strategy":"RandomDiscrete",                         
    'max_runtime_secs': 120, #15 min     
    'max_models': 100,  ## build no more than 100 models
    'seed' : 1
}

tree = H2OGradientBoostingEstimator()
xgb_grid = H2OGridSearch(
    model = tree, 
    hyper_params = hyper_params,
    grid_id = grid_id,
    search_criteria = search_criteria
)

xgb_grid.train(
    x = predictors,
    y = response,
    training_frame = h2o_x_train,
    validation_frame = h2o_x_test
)

In [ ]:
h2o.save_model(xgb_grid.models[0], path="h2o_models/xgb_frid", force=True)

In [ ]:
print(xgb_grid.model_performance())
print(xgb_grid.model_performance(h2o_test))
print(xgb_grid.r2())
print(xgb_grid.r2(valid=True))

In [ ]:
best_tree = xgb_grid.models[0]
print(best_tree.model_performance(h2o_x_test))
print(best_tree.r2())
print(best_tree.r2(valid=True))

In [ ]:
tree.varimp_plot()

In [ ]:
### Deep learning

In [ ]:
model = H2ODeepLearningEstimator()
model.train(
    x = predictors,
    y = response,
    training_frame = h2o_x_train,
    validation_frame = h2o_x_test
)
print(model.model_performance())
print(model.model_performance(h2o_test))
print(model.r2())
print(model.r2(valid=True))

In [ ]:
hyper_params = {
    "activation": ["Rectifier", "Maxout", "Tanh", "RectifierWithDropout", "MaxoutWithDropout", "TanhWithDropout"],
    "hidden": [(20,), (50,), (100,), (250,), (500,), (1000,), (2500,), (20, 20), (50, 50), (100, 100), (250, 250), (500, 500), (1000, 1000), (2500, 2500)],
    "epochs": [10, 25, 50, 100, 200, 500],
    "rho": [0.8, 0.9, 0.95, 0.99, 0.999],
    "rate": [0.5, 0.25, 0, 0.1, 0.005, 0.001]
    "momentum_start": [0, 0.25, 0.5, 1],
    "momentum_stable": [0, 0.25, 0.5, 1],
    "input_dropout_ratio ": [0, 0.1, 0.2, 0.5, 0.75],
    "epsilon": [1e-10, 1e-8, 1e-6, 1e-4]
    "max_w2": [10, 100, 1000, 3.4028235e+38],
    "l1": [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.25],
    "l2": [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.25],
    "seed": [1]
}
grid_id = 'max_depth_grid'
search_criteria = { 
    "strategy":"RandomDiscrete",                         
    'max_runtime_secs': 900, #15 min     
    'max_models': 100,  ## build no more than 100 models
    'seed' : 1
}

tree = H2ODeepLearningEstimator()
mlp_grid = H2OGridSearch(
    model = tree, 
    hyper_params = hyper_params,
    grid_id = grid_id,
    search_criteria = search_criteria
)

mlp_grid.train(
    x = predictors,
    y = response,
    training_frame = h2o_x_train,
    validation_frame = h2o_x_test
)